In [1]:
print("pio")

pio


In [6]:
!pip install fsspec

In [2]:
# %%
from config import model_name
import pandas as pd
from tqdm import tqdm
import os
import random
import numpy as np
import csv
import importlib
import copy
from collections import Counter
from datetime import timedelta
import sys

# A = 5 > 3
# B = 3 > 5
# print(A & B)
# print(A | B)

# print(os.getcwd())
# exit()


# config 가져오기
try:
    config = getattr(importlib.import_module('config'), f"{model_name}Config")
    
except AttributeError:
    print(f"{model_name} not included!")
    exit()

# 불필요한 경고를 무시
import warnings
warnings.filterwarnings('ignore')

# 문자열을 timestamp로 변환하는 함수
from datetime import datetime
def str_to_timestamp(string):
    return datetime.timestamp(datetime.strptime(string,'%Y-%m-%d %H:%M:%S'))

# 변수들 및 경로 설정
model_name = 'LSTUR' # NRMS에서 수정함
ratio = config.negative_sampling_ratio
lifetime = config.lifetime
preprocess_data_folder = './data/preprocessed_data'
preprocess_type = ''         # '(type1)'/ '(type2)' / ''
data_type = 'Adressa_4w'





# total_news(raw).tsv와 behaviors.tsv 경로 설정
print(f'\n{data_type} / {model_name} \nmake_behaviors_parsed')
news_dir = f'C://Users//Big Data Science LAB//seongju//datasets//{data_type}'
train_dir = f'C://Users//Big Data Science LAB//seongju//datasets//{data_type}/train'
# behaviors = pd.read_table(
#     os.path.join(train_dir,f'behaviors_parsed_ns{ratio}_lt{lifetime}.tsv'))
# behaviors['click'] = behaviors['candidate_news_impre'].str.split(' ')[0]


# behaviors.tsv 파일 불러오기
behaviors = pd.read_table(
    os.path.join(train_dir,'behaviors.tsv'),
    header=None,
    names=['user', 'time', 'history', 'click'])
# click 열의 값들을 '-'를 기준으로 나눔
behaviors.click = behaviors.click.str.split('-').str[0]
# nan은 빈 문자열 처리
behaviors.history.fillna(' ', inplace=True)

# %%
#### edited JW

# time열 datetime 자료형으로 변환
behaviors['time'] = pd.to_datetime(behaviors['time'])
##
added_columns_list = []

# total_news(raw).tsv 불러오기
news = pd.read_table(os.path.join(news_dir, 'total_news(raw).tsv'),
                    quoting=csv.QUOTE_NONE,
                    header=None,
                    names=['id','publish_time','title','clicks','category'])  # TODO try to avoid csv.QUOTE_NONE
"""바로 위 names 순서 고침"""

# category와 subcategory로 나누기
news[['category', 'subcategory']] = news['category'].str.split('|', expand=True)

# publish_time 열 datetime 자료형으로 변환
news['publish_time'] = pd.to_datetime(news['publish_time'])
# 뉴스 id를 index로 설정
news = news.set_index(['id'])
news.fillna(' ', inplace=True)
# news에 여러 종류의 클릭 수를 저장하는 새로운 columns 생성
news['train_click'] = 0
news['his_click'] = 0
news['lifetime_click'] = 0
news['current_lifetime_click'] = 0


# 클릭 열 카운트
train_click_cnt = behaviors['click'].value_counts()
# 유저 별 히스토리로 나눈 다음 {유저: 클릭 수} 형태로 dict에 저장
history_click = behaviors.groupby(by='user')['history'].last()
history_click_list = history_click.str.cat(sep=' ').split()
history_click_cnt = dict(Counter(history_click_list))
# train_click 열과 his_click 열에 위에서 구한 cnt 저장
for i,row in tqdm(news.iterrows(),desc="News training click count"):
    try:
        news.at[i,'train_click'] = train_click_cnt[i]
    except:
        pass

    try:
        news.at[i,'his_click'] = history_click_cnt[i]
    except:
        pass
    # print(type(row.publish_time))
    # print('row.publish_time', row.publish_time)
    # print(timedelta(hours=lifetime), '\n')
    # sys.exit()
    
    # end_time: 수명이 끝나는 시간
    end_time = row.publish_time + timedelta(hours=lifetime)
    # 유효한 수명 시간에 존재하는 클릭 기록들 추출
    behavior_mask = (behaviors['time']<= end_time) & (behaviors['time'] >= row.publish_time)
    behaviors_tmp = behaviors.loc[behavior_mask]
    # 추출한 애들 클릭 수 계산해서 dict에 저장
    click_cnt = dict(Counter(behaviors_tmp['click'].str.cat(sep=' ').split()))
    # 저장한 수명 클릭 수를 news의 lifetime_click 열에 저장
    try:
        news.at[i,'lifetime_click'] = click_cnt[i]
    except:
        pass
# news의 all_click 열은 behaviors의 뉴스 당 카운트 수와 유저 별 클릭 수의 합으로 저장
news['all_click'] = news['train_click'] + news['his_click']

# news의 lifetime_click 열의 값들을 내림차순으로 정렬
news = news.sort_values(by=['lifetime_click'], ascending=False)

# behaviors의 click 열을 유저 별로 나눠서 user_poslist에 저장
user_poslist = behaviors.groupby(by='user')['click'].apply(lambda x: ','.join(x))



Adressa_4w / LSTUR 
make_behaviors_parsed


News training click count: 59453it [22:05, 44.84it/s]


In [5]:
news.head()


,publish_time,title,clicks,category,subcategory,train_click,his_click,lifetime_click,current_lifetime_click,all_click
id,,,,,,,,,,
N5704,2017-02-09 07:39:00,Norsk landslagsprofil solgt til Kina for rekor...,"2017-02-10 08:00:02,2017-02-10 08:00:12,2017-0...",100sport,fotball,1,0,0,0,1
N45344,2017-03-03 14:27:47,FN bekymret for tortur-bruk på Sri Lanka,"2017-03-03 23:50:16,2017-03-03 23:51:41,2017-0...",nyheter,utenriks,0,0,0,0,0
N45332,2015-06-12 13:59:45,Godbiter fra Argentina,"2017-03-03 23:48:26,2017-03-04 04:26:52,2017-0...",kultur,vin,0,0,0,0,0
N45333,2012-10-05 11:31:59,- Hadde aldri vært der jeg er uten City Syd,"2017-03-03 23:48:31,2017-03-05 06:47:07",nyheter,trondheim,0,0,0,0,0
N45334,2016-01-10 19:00:05,Etter å ha trent minst fem timer dagen over en...,"2017-03-03 23:48:47,2017-03-07 06:52:56,2017-0...",100sport,sprek,0,0,0,0,0


In [6]:
print(behaviors.shape)
print(behaviors[:10])

(3125173, 4)
  user                time                                            history  \
0   U1 2017-02-10 08:00:02                                     N1 N1560 N1794   
1   U2 2017-02-10 08:00:02           N2 N89 N68 N1100 N1103 N12 N1085 N109 N6   
2   U3 2017-02-10 08:00:02                                            N3 N582   
3   U4 2017-02-10 08:00:02                 N1662 N1515 N1134 N4 N1725 N12 N16   
4   U5 2017-02-10 08:00:02                                            N630 N5   
5   U6 2017-02-10 08:00:02                                           N1134 N6   
6   U7 2017-02-10 08:00:03                                        N4 N36 N417   
7   U8 2017-02-10 08:00:03                                             N22 N5   
8   U9 2017-02-10 08:00:03                                                 N7   
9  U10 2017-02-10 08:00:03  N939 N7 N1636 N417 N1528 N1515 N1134 N1819 N17...   

   click  
0  N1895  
1  N1896  
2  N1897  
3  N1898  
4  N1899  
5  N1900  
6  N1898  
7  N189

In [ ]:
"""여기서 궁극적으로 하고 싶은 건 이 유저가 선택 안 한 애들 중에서, 다른 유저의 클릭 중 유효한 뉴스 수명을 가진 애만 남김
여기서, weight를 계산하는 것으로 인기도를 고려해서 인기 높은 뉴스를 negative sampling + 인기 낮은 뉴스를 negative sampling
그런 관점에서 아래 코드를 다시 보자"""
import time
from datetime import datetime

k = 1
for row in tqdm(behaviors[:1000000].itertuples(), desc=f"Generate {new_column_name} and {new_column_name2}"):
    """row: 'user', 'time', 'history', 'click' 순으로 저장된 한 유저의 클릭 정보
    여기서, 클릭한 뉴스는 오직 하나!
    타임은 그 뉴스를 클릭한 시간"""

    # 진행 상황 체크
    k += 1
    if k % 100000 == 0:
        print("k:", k)

        # 현재 Unix 타임스탬프를 연도, 날짜, 시간으로 변환
        timestamp = time.time()
        formatted_time = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')
        print("time:", time.time())

        # 현재까지의 데이터를 파일에 저장 (덮어쓰기)
        behaviors.to_csv(
                os.path.join(train_dir,f'behaviors_parsed_ns{ratio}_lt{lifetime}1.tsv'),
                sep='\t',
                index=False)
        print(f"파일이 저장되었습니다:", f'behaviors_parsed_ns{ratio}_lt{lifetime}1.tsv', '\n')
    
    
    endtime = row.time - timedelta(hours=lifetime)
    """poslist: 현재 행 (row)의 유저가 클릭한 뉴스 리스트"""
    poslist = user_poslist.loc[row.user].split(',')

    """여기서, mask의 역할: 수명 시간 내에 속한 뉴스만 골라냄
        mask 조건 해석: 뉴스 수명 한계 시간 <= 게시 시간 & 게시 시간 <= 클릭 시간
        mask: news의 모든 행에 대해 이런 형식 ([True, False, False, ..., True] )의 list"""
    mask = (news['publish_time'] <= row.time) & (news['publish_time'] >= endtime)
    """news_negative_candidate shape: (198,); 달리 말하면, 현재 시점 (row.time)에 대해 유효한 수명을 가진 모든 뉴스의 집합"""
    news_negative_candidate = copy.copy(news.loc[mask])
    behavior_mask = (behaviors['time']<= row.time) & (behaviors['time'] >= endtime)
    behaviors_tmp = behaviors.loc[behavior_mask] # 모든 유저의 히스토리 중 유효한 수명을 가진 히스토리만 남김; U1의 클릭에 대해서는 6개만 생존
    """click_cnt shape: (6,)"""
    click_cnt = dict(Counter(behaviors_tmp['click'].str.cat(sep=' ').split()))

    """click_cnt에 속하는 6개의 뉴스가 news_negative candidate에 없으면, current_lifetime_click의 값들은 모두 nan"""
    news_negative_candidate['current_lifetime_click'] = news_negative_candidate.index.map(click_cnt)
    news_negative_candidate.dropna(thresh=2,inplace=True) # debug 1. current_lifetime_click 확인
    
    """ ####################################### 내가 추가한 부분"""
    news_negative_candidate['current_lifetime_click'].fillna(0, inplace=True)

    news_negative_list = news_negative_candidate.index.to_list() # (198,)
    """news_weight shape: (198,)"""
    news_weight = news_negative_candidate.current_lifetime_click.to_list() 
    news_weight = [np.log(x+2) for x in news_weight] # debug 2. log 전 news_weight 확인

    for del_news in set(news_negative_list).intersection(poslist): # debug 3. log 후 news_weight 확인
        del_index = news_negative_list.index(del_news)

        del news_negative_list[del_index]
        del news_weight[del_index]
    """삭제 끝"""
    
    candidate_news = [row.click]

    if ratio > len(news_negative_list):
        times = (ratio//len(news_negative_list)+1)
        news_negative_list = news_negative_list*times

        news_weight = news_weight*times

    news_negative_list2 = copy.copy(news_negative_list)
    news_weight2 = [1/x for x in news_weight]
    candidate_news2 = copy.copy(candidate_news)

    for i in range(ratio): # debug 4. 코드 실행 전 news_weight 최종 확인
        ns = random.choices(news_negative_list,k=1,weights = news_weight) # error

        candidate_news.append(ns[0])
        del_index = news_negative_list.index(ns[0])

        del news_negative_list[del_index]
        del news_weight[del_index]

    for i in range(ratio):
        ns = random.choices(news_negative_list2,k=1,weights = news_weight2)

        candidate_news2.append(ns[0])
        del_index = news_negative_list2.index(ns[0])

        del news_negative_list2[del_index]
        del news_weight2[del_index]

    behaviors.at[row.Index,new_column_name] = ' '.join(candidate_news)
    behaviors.at[row.Index,new_column_name2] = ' '.join(candidate_news2)
    # tmp_cnt += 1
    # if tmp_cnt == 1000:
    #     break
    """여기까지가 row에 대한 for문"""
    
added_columns_list.append(new_column_name.replace('candidate_news_','',1))
added_columns_list.append(new_column_name2.replace('candidate_news_','',1))


# %%
behaviors['clicked'] = ' '.join(['1']+['0']*ratio)
del behaviors['click']
try:
    behaviors.rename(columns={'history': 'clicked_news'}, inplace=True)
except:
    pass
behaviors.to_csv(
        os.path.join(train_dir,f'behaviors_parsed_ns{ratio}_lt{lifetime}1.tsv'),
        sep='\t',
        index=False)

# 19시 50분: 첫 파일 저장 시간

Generate candidate_news_current_log_pop and candidate_news_rev_current_log_pop: 99997it [59:39, 19.07it/s]

k: 100000
time: 1731408653.1501272


Generate candidate_news_current_log_pop and candidate_news_rev_current_log_pop: 100001it [59:49,  1.04it/s]

파일이 저장되었습니다: behaviors_parsed_ns4_lt361.tsv 



Generate candidate_news_current_log_pop and candidate_news_rev_current_log_pop: 199997it [2:52:51, 11.94it/s]

...

Generate candidate_news_current_log_pop and candidate_news_rev_current_log_pop: 599998it [13:34:47, 11.44it/s]

k: 600000
time: 1731453960.427538


Generate candidate_news_current_log_pop and candidate_news_rev_current_log_pop: 600000it [13:34:57,  1.59s/it]

파일이 저장되었습니다: behaviors_parsed_ns4_lt361.tsv 



Generate candidate_news_current_log_pop and candidate_news_rev_current_log_pop: 690734it [16:07:13,  7.93it/s]

In [4]:
print(i)

4
